# Load Required Packages

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import scipy.stats as sc
# import shap
# import lime
import sklearn 
import warnings
#import xgboost
import itertools
import gc
import networkx as nx
import pydot

import multiprocessing as mp
import time
import pickle 
#import interpret
import collections
import math
import random
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import  RandomForestRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn import preprocessing
# from interpret.glassbox import ExplainableBoostingClassifier
# from interpret import show
from sklearn.preprocessing import LabelEncoder
# from interpret.glassbox import ExplainableBoostingRegressor
# from interpret import show
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import AdaBoostRegressor
%matplotlib inline
import os, sys
#import statsmodels.api as sm
sys.path.append(os.path.abspath("../../../"))

pd.set_option('display.max_columns', None)
import eli5
from eli5.sklearn import PermutationImportance

import sys
print(sys.path)

sys.path.append("/anaconda3/lib/python3.6/site-packages")
 

['/Users/aparnacalambur/Documents/Cornell/Classes/MEng/interpretable_clustering', '', '/Library/Python/3.6/site-packages', '/Users/aparnacalambur/Documents/Cornell/Classes/MEng/interpretable_clustering', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python37.zip', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload', '/Users/aparnacalambur/Library/Python/3.7/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions', '/Users/aparnacalambur/.ipython', '/Users/aparnacalambur/Documents/Cornell', '/anaconda3/lib/python3.6/site-packages', '/Users/aparnacalambur/Documents/Cornell']


In [10]:
income_data = pd.read_csv('adult.csv')
inc_no_tar = income_data.drop(columns = ['income'])
c = ['income'] + list(inc_no_tar.columns)
data = pd.DataFrame(columns=c)
for col in c:
    data[col] = income_data[col]
income_data = data.copy()

income_data['workclass']= income_data['workclass'].replace({"?":"Unknown"})
income_data['native-country']= income_data['native-country'].replace({"?":"Unknown"})
income_data = income_data.drop(['education', 'marital-status', 'fnlwgt'], axis=1)

label_encoder = preprocessing.LabelEncoder()
income_data['workclass']= label_encoder.fit_transform(income_data['workclass'])
income_data['occupation']= label_encoder.fit_transform(income_data['occupation'])
income_data['relationship']= label_encoder.fit_transform(income_data['relationship'])
income_data['race']= label_encoder.fit_transform(income_data['race'])
income_data['gender']= label_encoder.fit_transform(income_data['gender'])
income_data['native-country']= label_encoder.fit_transform(income_data['native-country'])
income_data['income']= label_encoder.fit_transform(income_data['income'])

full_data = income_data.sample(n=40000, random_state=1)

full_data_train,full_data_test = train_test_split(full_data,test_size = 0.3,random_state = 10)

full_data = full_data.dropna().reset_index().drop('index',axis = 1)
X = full_data.drop('income',axis = 1)
cols = X.columns
y = full_data['income']
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)
X =pd.DataFrame(X, columns = cols)
xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3,random_state = 10)
xTrain, xTest, yTrain, yTest, full_data_train,full_data_test  

(            age  workclass  educational-num  occupation  relationship  \
 33113 -0.630836  -0.228559         0.358098    1.517811      0.970670   
 23268 -0.484895  -0.228559        -0.030189   -0.608703     -0.900291   
 25562 -1.506484   2.554500        -0.030189   -1.553820      0.970670   
 28539  0.390752  -0.228559        -0.030189   -0.608703     -0.276637   
 7419  -0.703807  -2.315853        -0.418476   -1.317540     -0.276637   
 ...         ...        ...              ...         ...           ...   
 10201  0.390752  -0.228559        -0.418476   -0.844982     -0.276637   
 9372  -1.360543   2.554500        -0.418476   -1.553820      0.970670   
 28017 -1.141631  -0.228559         1.134672   -0.608703     -0.276637   
 29199  0.609664  -0.228559        -0.418476    1.281531     -0.900291   
 17673  0.755605  -0.228559        -0.418476   -0.844982     -0.900291   
 
            race    gender  capital-gain  capital-loss  hours-per-week  \
 33113  0.392814 -1.423117     -0.14

# Load Test Data

In [11]:
titanic = pd.read_csv('train.csv')
#titanic
full_data = titanic
full_data = full_data.drop(['PassengerId'], axis=1)

import re
deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
full_data['Cabin'] = full_data['Cabin'].fillna("U0")
full_data['Deck'] = full_data['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
full_data['Deck'] = full_data['Deck'].map(deck)
full_data['Deck'] = full_data['Deck'].fillna(0)
full_data['Deck'] = full_data['Deck'].astype(int)

full_data = full_data.drop('Cabin', axis = 1)

mean = full_data["Age"].mean()
std = full_data["Age"].std()
is_null = full_data["Age"].isnull().sum()
# compute random numbers between the mean, std and is_null
rand_age = np.random.randint(mean - std, mean + std, size = is_null)
# fill NaN values in Age column with random values generated
age_slice = full_data["Age"].copy()
age_slice[np.isnan(age_slice)] = rand_age
full_data["Age"] = age_slice
full_data["Age"] = full_data["Age"].astype(int)
full_data["Age"].isnull().sum()
full_data['Embarked'] = full_data['Embarked'].fillna('S')


full_data['Fare'] = full_data['Fare'].fillna(0)
full_data['Fare'] = full_data['Fare'].astype(int)
full_data = full_data.drop(['Name'], axis=1)
full_data = full_data.drop(['Ticket'], axis=1)
full_data['Sex'] = full_data['Sex'].map({"male": 0, "female": 1})


full_data_train,full_data_test = train_test_split(full_data,test_size = 0.3,random_state = 10)

full_data = pd.get_dummies(full_data,columns = ['Pclass','Embarked','Deck'])
full_data = full_data.dropna().reset_index().drop('index',axis = 1)
X = full_data.drop('Survived',axis = 1)
cols = X.columns
y = full_data['Survived']
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)
X =pd.DataFrame(X, columns = cols)
xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3,random_state = 10)


In [12]:
heart = pd.read_csv("heart.csv")
#full_data = heart
heart = pd.read_csv("heart.csv")
heart_no_tar = heart.drop(columns = ['target'])
c = ['target'] + list(heart_no_tar.columns)
full_data = pd.DataFrame(columns=c)
for col in c:
    full_data[col] = heart[col]

full_data_train,full_data_test = train_test_split(full_data,test_size = 0.3,random_state = 10)

full_data = full_data.dropna().reset_index().drop('index',axis = 1)
X = full_data.drop('target',axis = 1)
cols = X.columns
y = full_data['target']
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)
X =pd.DataFrame(X, columns = cols)
xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3,random_state = 10)
xTrain, xTest, yTrain, yTest, full_data_train,full_data_test  

(          age       sex        cp  trestbps      chol       fbs   restecg  \
 49  -0.150692 -1.468418 -0.938515  0.364165 -0.237008 -0.417635 -1.005832   
 171 -0.702136  0.681005  0.032031 -1.234996 -0.333636 -0.417635  0.898962   
 223  0.180175 -1.468418 -0.938515  3.905165  0.806568  2.394438 -1.005832   
 58  -2.246179  0.681005  1.973123 -0.778093 -1.241933 -0.417635 -1.005832   
 154 -1.694735 -1.468418  1.002577  0.364165 -0.507565 -0.417635  0.898962   
 ..        ...       ...       ...       ...       ...       ...       ...   
 156 -0.812425  0.681005  1.002577 -0.092738  0.130176 -0.417635  0.898962   
 123 -0.040403 -1.468418  1.002577 -1.349222  0.400733 -0.417635 -1.005832   
 15  -0.481558 -1.468418  1.002577 -0.663867 -0.526890 -0.417635  0.898962   
 125 -2.246179 -1.468418  0.032031 -0.778093 -0.700820 -0.417635  0.898962   
 265  1.283063  0.681005 -0.938515 -1.120770 -0.662169 -0.417635 -1.005832   
 
       thalach     exang   oldpeak     slope        ca      th

In [13]:
income_data = pd.read_csv('adult.csv')
inc_no_tar = income_data.drop(columns = ['income'])
c = ['income'] + list(inc_no_tar.columns)
data = pd.DataFrame(columns=c)
for col in c:
    data[col] = income_data[col]
income_data = data.copy()

income_data['workclass']= income_data['workclass'].replace({"?":"Unknown"})
income_data['native-country']= income_data['native-country'].replace({"?":"Unknown"})
income_data = income_data.drop(['education', 'marital-status', 'fnlwgt'], axis=1)

label_encoder = preprocessing.LabelEncoder()
income_data['workclass']= label_encoder.fit_transform(income_data['workclass'])
income_data['occupation']= label_encoder.fit_transform(income_data['occupation'])
income_data['relationship']= label_encoder.fit_transform(income_data['relationship'])
income_data['race']= label_encoder.fit_transform(income_data['race'])
income_data['gender']= label_encoder.fit_transform(income_data['gender'])
income_data['native-country']= label_encoder.fit_transform(income_data['native-country'])
income_data['income']= label_encoder.fit_transform(income_data['income'])

full_data = income_data.sample(n=5000, random_state=1)

full_data_train,full_data_test = train_test_split(full_data,test_size = 0.3,random_state = 10)

full_data = full_data.dropna().reset_index().drop('index',axis = 1)
X = full_data.drop('income',axis = 1)
cols = X.columns
y = full_data['income']
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)
X =pd.DataFrame(X, columns = cols)
xTrain, xTest, yTrain, yTest = train_test_split(X,y, test_size = 0.3,random_state = 10)
xTrain, xTest, yTrain, yTest, full_data_train,full_data_test  

(           age  workclass  educational-num  occupation  relationship  \
 2573  0.006122  -0.224168        -0.419627    1.754091     -0.892873   
 2066  0.006122   1.153631         1.124824    0.809279     -0.892873   
 2823  1.094159  -0.224168         0.738711    1.281685     -0.892873   
 2389  0.078658  -0.224168         0.738711    1.045482     -0.892873   
 2702 -0.719236   0.464732         1.124824    0.809279     -0.268748   
 ...        ...        ...              ...         ...           ...   
 1180  0.368801  -0.224168         1.510937   -0.607939     -0.892873   
 3441  0.078658  -0.224168        -0.033515   -1.316549     -0.268748   
 1344  0.078658  -0.224168         0.738711    1.281685     -0.892873   
 4623  0.876551   2.531430        -0.419627   -1.552752      1.603627   
 1289  0.078658  -0.224168        -0.033515    1.754091     -0.268748   
 
           race    gender  capital-gain  capital-loss  hours-per-week  \
 2573  0.398681  0.702127     -0.152843     -0.22

https://scikit-learn.org/dev/auto_examples/tree/plot_unveil_tree_structure.html

In [14]:
def compute_distances(tree1, X):
    n_nodes = tree1.tree_.node_count
    children_left = tree1.tree_.children_left
    children_right = tree1.tree_.children_right
    feature = tree1.tree_.feature
    threshold = tree1.tree_.threshold

    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
    while len(stack) > 0:
        node_id, depth = stack.pop()
        node_depth[node_id] = depth
        is_split_node = children_left[node_id] != children_right[node_id]
        if is_split_node:
            stack.append((children_left[node_id], depth + 1))
            stack.append((children_right[node_id], depth + 1))
        else:
            is_leaves[node_id] = True

    leaves = [i for i, x in enumerate(is_leaves) if x]
    leaves = [str(l) for l in leaves]
    leaf_pairs = list(itertools.combinations(leaves,2))
    uMG = dt_to_mg(tree1)
    res = []
    for inds in leaf_pairs:
        source = inds[0]
        target = inds[1]
        distance = nx.shortest_path_length(uMG, source=inds[0], target=inds[1])
        res.append([(source+','+target),distance])
    results = pd.DataFrame(res,columns = ['key','dist'])

    leaf_id = tree1.apply(X)
    instance_pairs = list(itertools.combinations(range(0,len(leaf_id)),2))
    instance_dist = pd.DataFrame(instance_pairs,columns = ['i1','i2'])
    instance_dist['l1'] = leaf_id[instance_dist['i1'].values]
    instance_dist['l2'] = leaf_id[instance_dist['i2'].values]
    instance_dist['leaf1'] = instance_dist[['l1','l2']].min(axis = 1)
    instance_dist['leaf2'] = instance_dist[['l1','l2']].max(axis = 1)
    instance_dist = instance_dist.drop(['l1','l2'], axis = 1)
    instance_dist['key'] = instance_dist['leaf1'].astype(str) + ',' + instance_dist['leaf2'].astype(str)
    all_distances = pd.merge(instance_dist,results,on = 'key', how = 'left')
    all_distances = all_distances.fillna(0)

    return all_distances[['i1','i2','dist']]

def forest_distances(rf,X):
    alldist = pd.DataFrame()
    for tree1 in rf.estimators_:
        dist = compute_distances(tree1, X)
        alldist = alldist.append(dist)
    alldist_agg = alldist.groupby(['i1','i2']).dist.mean().reset_index()
    
    s = max(max(alldist_agg['i1']), max(alldist_agg['i2'])) + 1
    distmatrix = np.zeros(shape = (s,s))
    for i,row in alldist_agg.iterrows():
        i1 = int(row['i1'])
        i2 = int(row['i2'])
        distmatrix[i1][i2] = row['dist']
        distmatrix[i2][i1] = row['dist']

    return distmatrix

def dt_to_mg(clf):
    dot_data = tree.export_graphviz(clf)
    dot_graph = pydot.graph_from_dot_data(dot_data)[0]
    MG = nx.drawing.nx_pydot.from_pydot(dot_graph)
    uMG = MG.to_undirected()
    return uMG

In [15]:
rf = RandomForestClassifier(n_estimators = 5, max_depth = 4).fit(xTrain,yTrain)
pred = rf.predict(xTest)
print(sklearn.metrics.roc_auc_score(pred,yTest))

0.8542367959024725


In [16]:
rf = RandomForestClassifier(n_estimators = 5, max_depth = 4).fit(xTrain,yTrain)
pred = rf.predict(xTest)
distances = forest_distances(rf,xTrain)

In [17]:
distances

array([[0. , 6.6, 2.4, ..., 2.8, 5.6, 2.4],
       [6.6, 0. , 5.4, ..., 7. , 7.4, 7. ],
       [2.4, 5.4, 0. , ..., 1.6, 6. , 4. ],
       ...,
       [2.8, 7. , 1.6, ..., 0. , 5.6, 4.4],
       [5.6, 7.4, 6. , ..., 5.6, 0. , 4. ],
       [2.4, 7. , 4. , ..., 4.4, 4. , 0. ]])

In [18]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(n_clusters = 2).fit(distances)
xTrain['clust'] = clustering.labels_
clusters = pd.DataFrame(np.column_stack((yTrain,clustering.labels_)),columns = ['y','clust'])
clusters.groupby('clust').mean()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/scipy/cluster/hierarchy.py:830: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  return linkage(y, method='ward', metric='euclidean')
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,y
clust,
0,0.458247
1,0.089225


In [26]:
len(distances)

212

In [27]:
from itertools import product

In [28]:
inds = [i for i in range(0, len(distances))] 
tuples = list(product(inds, inds))
dist = [j for sub in distances for j in sub]
train_final_dist_dct = dict(zip(tuples, dist)) 
ind_tup_dct = {}
for i, j in tuples:
    if i in ind_tup_dct.keys():
        ind_tup_dct[i].add((i, j))
    else:
        ind_tup_dct[i] = set()
        ind_tup_dct[i].add((i, j))
    if j in ind_tup_dct.keys():
        ind_tup_dct[j].add((i, j))
    else:
        ind_tup_dct[j] = set()
        ind_tup_dct[j].add((i, j))